### Test BIT* Planner on Static Rand Maze2D Env with Concave Obstacles

In [ ]:
import time, sys
sys.path.append('..')
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
from utils.maze2d_utils import plt_img, save_img
from pb_diff_envs.environment.static.maze2d_rand_wgrp import Maze2DRandRecGroupList
from pb_diff_envs.environment.static.rand_maze2d_env import RandMaze2DEnv

no_while = True
# --- 2 ---
## avg one traj len: , horizon = 
num_groups = 5
num_walls = 6
maze_size = np.array([5, 5])
hr = 0.50
hExt_range = [hr, hr] # fixed-size obstacle
seed = 333

hExt_range = np.array(hExt_range)

trivial = dict(gen_data=True, 
               samples_per_env=200, gen_num_parts=1, 
               eval_num_groups=6, eval_num_probs=10,
               dataset_url='./dataset/luotest-randmaze2d-ipynb.hdf5')

mazelist_config = dict(planner_timeout=30, 
                        interp_density=10,
                        gap_betw_wall=0.15,
                        min_to_wall_dist=0.02,
                        )
                        # min_episode_distance=)

import gym
env_name = 'randSmaze2d-C43-luotest-v0' # 100 prob: d8 (897, 2) -> (3459, 2)
# env_name = 'randSmaze2d-C43-ng3ks25k-ms55nw21-hExt05-v0'
# env_name = 'randSmaze2d-C43-ng3ks25k-ms55nw21-hExt05Rce01-v0'
dvg_group = gym.make(env_name, load_mazeEnv=False, gen_data=True, is_eval=True) # True False


In [ ]:
## some env group stats
dvg_group.wallLoc_list.shape
dvg_group.recs_grp.save_center_and_mode()
dvg_group.recs_grp.centers_43[1]
dvg_group.recs_grp.mode_list

In [ ]:
## each concave obstacles consists of several small square blocks
env = dvg_group.create_single_env(2)
env.load(GUI=True)
prev_pose = env.get_robot_free_pose()
## solve motion planning task using BIT*
solution, _ = env.sample_1_episode(prev_pose)
print('solution', solution)
env.robot.set_config(prev_pose)
env.wall_locations

## Vis 1 traj

In [ ]:
from utils.kuka_utils_luo import SolutionInterp
import matplotlib.pyplot as plt

sol_interp = SolutionInterp(density=10)
traj_vis = sol_interp(solution)
print('solution', len(solution))
solution_vis = np.array(solution)
print('traj_vis', len(traj_vis))
savepath = None
img = env.render_1_traj(savepath, traj_vis[-70:],)
plt_img(img)

## Vis many trajs

In [ ]:
from utils.utils import DotDict
from tqdm import tqdm
from utils.kuka_utils_luo import SolutionInterp
import matplotlib.pyplot as plt
sol_interp = SolutionInterp(density=8)
env.load(GUI=True) # False
prev_pose = env.get_robot_free_pose()
env.robot.set_config(prev_pose)
solutions = []
# n_traj = 25
n_traj = 5
for i in tqdm(range(n_traj)):
    solution, _ = env.sample_1_episode(prev_pose)
    solution = np.array(solution)
    prev_pose = solution[-1]
    solutions.append(solution)

trajs = solutions
solutions = np.concatenate(solutions, axis=0)
result = DotDict(solution=solutions)


print(solutions.shape)
print(len(result.solution))
# result

In [ ]:
import matplotlib.pyplot as plt
solution = np.array(result.solution)
# env.render_1_traj(solution)
for i in tqdm(range(n_traj)):
    trajs[i] = sol_interp(trajs[i])
savepath = None
tmp = env.render_composite(savepath, trajs)
print(tmp.shape)
# plt.clf()
# plt.imshow(tmp)
# plt.show()
plt_img(tmp, dpi=200)
